# 4. 쳇봇엔진에 필요한 딥러닝 모델

## 4.1 Keras

* 케라스는 신경망모델 구축시 필요한 라이브러리이다.
* 설치 : pip install tensorflow

In [ ]:
# MNIST분류모델학습
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test /  255.0 # 0~1사이의 값으로 정규화

In [ ]:
# tf.data를 이용하여 데이터셋을 셔플링
ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000)
train_size = int(len(X_train) * 0.7) # 학습용 : 검증용 = 7:3
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).batch(20)

In [ ]:
# MNIST분류모델
# 캐라스에서 모델을 만드는 방법은 순차모델, 함수형모델 2가지가 있다.
# 1. 모델정의
model = Sequential() # 순차모델
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(20, activation="relu"))
model.add(Dense(20, activation="relu"))
model.add(Dense(10, activation="softmax"))

# 2. 모델생성
# 오차계산함수 : sparse_categorical_crossentropy
# 오차를 보정 옵티마이저 : sgd
# 측정항목 : 정확도로 설정 accuracy
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

# 3. 모델학습
hist = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
# 4. 모델평가
model.evaluate(X_test, y_test)
model.summary()

# 5. 모델저장
model.save('./data/chatbot/mnist_modle.h5')

In [ ]:
# 시각화
fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()
loss_ax.plot(hist.history['loss'], 'y', label="train loss")
loss_ax.plot(hist.history['val_loss'], 'r', label="val loss")
acc_ax.plot(hist.history['accuracy'], 'y', label="train accuracy")
acc_ax.plot(hist.history['val_accuracy'], 'r', label="val accuracy")
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')
loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')
plt.show()

In [ ]:
# 학습된 모델 사용해서 예측하기
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
_, (X_test, y_test) = mnist.load_data()
X_test = X_test / 255.0

model = load_model('./data/chatbot/mnist_modle.h5')
model.summary()
model.evaluate(X_test, y_test, verbose=2)


In [ ]:
plt.imshow(X_test[20], cmap="gray")
plt.show()

In [ ]:
# 20번째이미지 분류하기
import numpy as np
img = [20]
predict = np.argmax(model.predict(X_test[img]), axis=1)
print('손글씨 이미지의 예측 값 = ', predict)

## 4.2 CNN

* 컴퓨터비젼분야에 대표적으로 사용되는 모델로서 이미지 분류에 좋은 성능을 가지고 있다.
* 얼굴인식, 이미지판별, 자율주행분야에 많이 활용

### 4.2.1 CNN 개념

* 합성곱(Convolution)과 풀링(Pooling)연산에 대해 알야야 한다.
* 합성곱연산은 필터(filter)라고 하는 특정크기의 행렬을 소스(이미지행렬, 문장행렬등..)와 곱하거나 더하는 연산을 수행
* 합성곱필터는 경우에 따라 `마스크 mask, 윈도우 window, 커널 kernel`등으로 다향하게 불리지만 보통 필터 or 커널로 호칭한다.
* 필터의 위치를 몇 칸을 이동할지 결정하는 값을 `스트라이드 stride`라고 한다.
* 연산을 거칠때 마다 특징맵(연산후 결과를 feature map이라고 한다)의 크기가 작아 지는데 이를 방지하기 위해 `패딩 padding`dmf tkdydgksek.
* `패딩과 스트라이드에 따른 출력크기는 Keras가 알아서 계산`해주기 때문에 개념만 알고 있으면 된다.

### 4.2.2 CNN 모델로 문장데이터의 감정분류 모델 구현

* chatbot_data.csv의 구조
|Q|A|Label|
|:------|:----------|:----:|
|12시 땡!|하루가 또 가네요.|0|
|1지망 학교 떨어졌어|위로해 드립니다.|0|
|3박4일 놀러가고 싶다|여행은 언제나 좋죠.|0|
|3박4일 정도 놀러가고 싶다|여행은 언제나 좋죠.|0|
|PPL 심하네|눈살이 찌푸려지죠.|0
|SD카드 망가졌어|다시 새로 사는 게 마음 편해요.|0|
|SD카드 안돼|다시 새로 사는 게 마음 편해요.|0|

* 분류
|Label|의미|
|:----:|:---------|
|0|일상다반사|
|1|이별(부정)|
|2|사랑(긍정)|


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [ ]:
# 1. 데이터읽어오기
train_file = "./data/chatbot/chatbot_data.csv"
data = pd.read_csv(train_file)
data.head()
features = data.Q.tolist()
labels = data.label.tolist()
print(features[:5], labels[:5])
print(features[-10:], labels[-10:])

In [ ]:
# 2. 단어인덱스시퀀스벡터생성
# 질문리스트(features)에서 문장(text)을 하나씩 꺼내서 text_to_word_sequence(text)함수로
# 단어들의 시퀀스를 생성
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
print(type(corpus), corpus[:5], '\n총건수 = ', len(corpus))

In [ ]:
# 3. 토크나이징 & 단어시퀀스
tokenizer = preprocessing.text.Tokenizer()
print(type(tokenizer))
tokenizer.fit_on_texts(corpus)

sequences = tokenizer.texts_to_sequences(corpus)
print(type(sequences), sequences[:5]) 

word_index = tokenizer.word_index 
print(type(word_index), word_index['회사'])

# 시퀀스번호로 생성된 벡터크기가 제각각이기 때문에 CNN의 입력계층을 고정된 입력노드로 설정
MAX_SEQ_LEN = 15

# 고정된 크기보다 작은 벡터에 남은 공간을 0으로 채우는 padding처리과정
# maxlen를 너무 크게 잡으면 메모리낭비, 너무 작게 잡으면 데이터손실이 발생
# padding : pre - 데이터앞에 패딩처리, post - 데이터뒤에 패딩처리
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(type(padded_seqs), padded_seqs[:1])

In [ ]:
# 4. 학습용, 검증용, 데스트용 데이터셋 생성 (7:2:1)
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
print(type(ds), ds)

train_size = int(len(features) * 0.7)
val_size = int(len(features) * 0.2)
test_size = int(len(features) * 0.1)
print(len(features), train_size, val_size, test_size)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

print(type(train_ds), train_ds)

In [ ]:
# 5. CNN모델을 정의
# 1) 하이퍼파라미터(옵션)을 정의
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체단어수

# 2) CNN모델정의 - 케라스 함수형 모델방식으로 구현
#    전처리된 입력데이터를 단어 임베딩처리영역과 연산을 통해 문장의 특징맵을 추출하고
#    평탄화를 거쳐서 완전연결계층 fully connection layer를 통해 감정별로 분류하는 영역으로 구서
# a. 입력계층
input_layer = Input(shape=(MAX_SEQ_LEN,))
print(type(input_layer), input_layer)

# b. 임베딩계층
#    단어별로 패딩처리된 시퀀스벡터(희소벡터)를 입력을 받아서 데이터손실을 최소화하면거
#    벡터차원이 압축되는 밀집벡터로 변환작업
#    단어의 갯수(VOCAB_SIZE)와 임베딩결과출력의 밀집벡터의 크기(EMB_SIZE), 입력되는 시퀀스
#    벡터의 크기(MAX_SEQ_LEN)를 기준으로 임베딩계층을 생성
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)

# c. 과적합을 방지하기 의해 dropout계층
#    dropout_prob 즉 비율이 50%만큼 dropout
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# d. 임베딩계층에서 전달된 임베딩벡터의 특징을 추출하는 계층
#    커널크기가 3, 4, 5인 합성곱필터를 통해 최대풀링을 통해 계층
conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# e. n-gram(3,4,5)이후 병합 - 특징맵의 결과를 하나로 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)

# f. 마지막으로 sofrmax함수를 통해서 감정별확률을 계산
predictions = Dense(3, activation=tf.nn.softmax)(logits)

# g. 모델을 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# h. 모델을 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# i. 모델을 평가(테스트데이터셋)
loss, accuracy = model.evaluate(test_ds, verbose=1)

# j. 모델을 저장
model.save("./data/chabot/cnn_model.h5")
# model.save("./data/chatbot/cnn_model.keras")

### 4.2.3 훈련된 CNN모델 사용

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [ ]:
# 1. 데이터읽기
train_file = "./data/chatbot/chatbot_data.csv"
data = pd.read_csv(train_file)
data.head()
features = data.Q.tolist()
labels = data.label.tolist()

# 2. 단어인덱스시퀀스벡터생성
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 3. 테스트데이터셋생성
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20)

# 4. 훈련된 모델불러오기
model = load_model("./data/chabot/cnn_model.keras")
model.summary()

# 5. 평가
model.evaluate(test_ds, verbose=1)

In [ ]:
# 6. 임의의 데스트데이터를 학습된 모델로 예측하기

# 1) 데스트데이터선택
sts = 3213
print('단어 시퀀스 : ', corpus[sts])
print('단어 인덱스 시퀀스 : ', padded_seqs[sts])
print('문장분류(정답) : ', labels[sts])

# 2) 예측
picks = [sts]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정예측점수 = ", predict)
print("감정예측분류 = ", predict_class.numpy())

## 4.3 개체명인식을 위한 양방향 LSTM모델

* LSTM은 순환신경망모델의 일종으로 시퀀스 or 시계열데이터의 패턴을 인식하는 분야에 많이 사용한다.

### 4.3.1 RNN

* LSTM은 RNN(Recurrent Neural Network)에서 파생된 모델
* RRN순환신경망으로 불리는데 은닉층노드의 출력값을 출력층과 다음의 은닉층노드의 입력으로 전달해서 순환하는 특징이 있다.

><img src="./images/0604.RNN_01.png" width="300" height="200" />
><img src="./images/0604.RNN_02.png" width="300" height="200" />
><img src="./images/0604.RNN_03.png" width="300" height="200" />
><img src="./images/0604.RNN_04.png" width="300" height="200" />
><img src="./images/0604.RNN_05.png" width="300" height="200" />
><img src="./images/0604.RNN_06.png" width="300" height="200" />
><img src="./images/0604.RNN_07.png" width="300" height="200" />
><img src="./images/0604.RNN_08.png" width="300" height="200" />
><img src="./images/0604.RNN_09.png" width="300" height="200" />
><img src="./images/0604.RNN_10.png" width="300" height="200" />
><img src="./images/0604.RNN_11.png" width="300" height="200" />


In [ ]:
# sin곡선을 RRN으로 예측
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, LSTM, SimpleRNN

In [ ]:
# time step만큼 시퀀스데이터를 생성(분리)함수
def split_sequence(sequence, step):
    X, y = list(), list()
    
    for i in range(len(sequence)):
        end_idx = i + step
        if end_idx > len(sequence) - 1:
            break
            
        seq_x, seq_y = sequence[i:end_idx], sequence[end_idx]
        X.append(seq_x)
        y.append(seq_y)
        
    return np.array(X), np.array(y)

In [ ]:
# 1. sin 학습데이터
# -10~10까지의 x축을 가지는 sin()함수의 값을 0.1씩 증가호 train_y에 저장
X = [i for i in np.arange(start=-10, stop=10, step=0.1)]
train_y = [np.sin(i) for i in X]
print(X[:5], '\n', train_y[:5])

In [ ]:
# 2. 하이퍼파라미터설정
# RNN의 입력시퀀스길이를 15로 정의 -> n_timesteps수 만큼 RNN셀을 생성
# 입력데이터의 차원(벡터) -> n_features = 1
n_timesteps = 15
n_features = 1

# 시퀀스나누기
# RRNN모델의 입력시퀀스를 나누기 위해 split_sequences()함수를 호출
# sin파형의 학습데이터 train_y에서 입력시퀀스길이 만큼 나눠서 입력시퀀스를 생성
X_train, y_train = split_sequence(train_y, step=n_timesteps)
print(f'shape X_train:{X_train.shape}, shape y_train:{y_train.shape}')

In [ ]:
# 3. RNN입력벡터크기를 맞추기 위해 벡터차원의 크기를 변경
#    keras.RRN계층을 사용하려면 3차원텐서형태이어야 한다.
#    따라서 2차원인 X_train의 차원을 RRN모델의 입력형태에 맞게 3차원(배치크기, 타임스템, 입력길이)
#    으로 변환 -> reshape함수를 이용
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], n_features)
print(f'shape X_train:{X_train.shape}, shape y_train:{y_train.shape}')

In [ ]:
# 4. RNN모델을 정의
#    sin파형의 데이터셋을 학습하기 위한 RRN계층을 정의한 후에 모델을 생성
#    모델 = SimpleRNN + Dense계층으로 구성
#    SimpleRNN은 가장 간단한 RNN계층
#    ... units=10 -> RNN계층에 존재하는 전체 뉴런의 갯수
#    ... return_sequences=False -> RRN계산과정에서 은닉상태값의 출력여부
#        -> False : 마지막 셀결과만 출력
#        -> True  : 모든 RNN계산과정의 결과를 출력
#        -> 이 옵션은 one-to-many, many-to-many구조를 위해 사용
#    ... input_shape=(n_timesteps, n_features) -> 입력데이터의 크기를 설정
model = Sequential()
model.add(SimpleRNN(units=10, return_sequences=False, input_shape=(n_timesteps, n_features)))
model.add(Dense(1))
# 손실함수=mse(정답과 오차의 차이), 옵티마이저(기울기조정)함수= adam
model.compile(optimizer='adam', loss='mse')

In [ ]:
# 5. 모델학습
# EarlyStopping콜백객체를 사용해서 손실의 변동이 없거나 갑자기 증가하는 시점에 조기중단
np.random.seed(42)
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=5, mode="auto")
history = model.fit(X_train, y_train, epochs=1000, callbacks=[early_stopping])

In [ ]:
# 6. loss시각화
plt.plot(history.history['loss'], label="loss")
plt.legend(loc='upper right')
plt.show()

In [ ]:
# 7. 학습된 RNN모델로 예측
# 1) 테스트데이터셋생성
#    ... 10~20사이의 x축범위를 가지는 cod()함수값을 0.1단위로 증가후에 y_calc에 저장
#    ... y_calc에 RNN모델을 테스트하기 위한 전체 시퀀스값을 저장
#    ... cos()함수를 이용하는 이유는 학습된 sin파형과 주기적인 차이를 전달하기 위해 임의로 생성
X_test = np.arange(10, 20, 0.1)
y_calc = np.cos(X_test)

# 2) RNN모델로 예측 및 로그를 저장
y_test = y_calc[:n_timesteps]

for i in range(len(X_test) - n_timesteps):
    net_input = y_test[i:i+n_timesteps]
    net_input = net_input.reshape((1, n_timesteps, n_features))
    y_train = model.predict(net_input, verbose=0)
    # print(y_test.shape, y_train.shape, i, i+n_timesteps)
    y_test = np.append(y_test, y_train)    

In [ ]:
# 3) 예측결과 시각화
plt.plot(X_test, y_calc, label="Real Value", color="orange")
plt.plot(X_test, y_test, label="Prediction Value", color="blue")
plt.legend(loc="upper right")
plt.ylim(-2, 2)
plt.show()

### 4.3.2 LSTM

* RNN모델은 입력시퀀스(층의 깊이)가 길어질 수록 앞쪽의 데이터가 뒤로 잘 전달되지 않아 학습능력이 저하
* 이런 문제를 해결하기 위해 RNN을 변형한 `LSTM Long Short Term Memory`를 개발

><img src="./images/0605.LSTM_01.jpg" width="300" height="200" />

### 4.3.3 양방향 LSTM

* RNN과 LSTM은 구조상 데이터가 입력된 순으로 처리되기 때문에 이전 시점의 정보만 활용가능하다는 단점이 있다
* 따라서, 문장이 길어질 수록 성능이 저하될 수 밖에 없다.
* 이런 문제를 해결하기 위해서 양방향LSTM(Bidirectional LSTM)모델이 개발 되었다.
* Bi-LSTM은 기존계층에 `역방향처리를 위한 LSTM계층을 하나 더 추가해서 양방향으로 문장의 패턴을 분석`
* 입력문장을 양방에서 처리하기 때문에 시퀀스의 길이가 길어진다 해도 `정보손실없이 처리가 가능`하다.
><img src="./images/0605.LSTM_02.png" width="400" height="300" />

### 4.3.4 개체명인식

* 임의의 문장에서 각 객체의 유형을 인식하는 것을 `개체명 인식 NER, Named Entity Recongniztion`라고 한다.
* NER이란 `문장에 포함된 단어가 인물, 조직, 장소등을 의미하는 단어인지를 인식하는 것`을 말한다.
* 딥러닝모델이나 확률모델등을 이용해서 문장내에서 개체명을 인식하는 것을 `개체명인식기`라고 한다.
* 문장을 정확하게 인식하기 위해서는 `반드시 처리해야 하는 전처리과정`이다.
* 개체명사전구축이 필요하다. 게다가 신조어, 사전미포함단어등의 처리는 불가능하기 때문에 사람이 직접 사전을 구축해야 되기 때문에 비용이 많이 든다.
* 개체명 인식모델을 만들기 위해서는 `BIO Beginning Inside Outside 표기법`을 알아야 한다.
  - B : 개체명이 시작되는 단어에 `B-개체명`으로 태그
  - I : B-개체명과 연결되는 단어일 때 `I-개체명`으로 태그
  - O : B,I를 제외한 모든 토킁에 태그
  >$$BIO표기예제$$
  >* 오늘부터 홍 길동은 삼성전자에 근무합니다.
  >* 오늘:B-Date, 홍:B-Person, 삼성:B-Company 근무:O
  >* 부터:O       길동:I-Person 전자:I-Company 합니다:O
  >* .            은:O           에: O
  
* 개체명인식모델을 학습하기 위해서는 `토큰별로 BIO태그가 달린 데이터셋이 필요`하다.
* 한글인 경우에 BIO태그사전을 구하는 것이 어렵지만 `국립국어원 언어정보나눔터에서 말뭉치를 공개`했다.
  - HLCT2016에서 제공하는 말뭉치를 수정한 KoreanNERCorpus:     
  - 다운로드 : http://github.com/machinereading/KoreanNERCorpus
  - train.txt데이터셋
    - `;로 시작하는 문장라인은 원본문장`
    - `$로 시작하는 문장라인은 해당 문장에서 NER처리된 결과`를 의미
    - 그 다음라인부터는 토큰번호, 단어토큰, 품사태그, BIO태그로 구성된 열
    - 본 강의에서는 `단어토큰고 BIO태그정보만 학습데이터셋으로 사용`
    
* NER인식에 필요한 시퀀스관리를 하는 라이브러리 추가
  - `pip install seqeval`
  - https://pypi.org/project/seqeval/ 
  - sequence labeling evaluation 라이브러리

In [1]:
# 양방향 LSTM
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import preprocessing
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, TimeDistributed

In [13]:
# 1. 학습파일로딩함수
def read_file(file_name):
    sents = []
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for idx, l in enumerate(lines):
            if l[0] == ';' and lines[idx + 1][0] == '$':
                this_sent = []
            elif l[0] == '$' and lines[idx - 1][0] == ';':
                continue
            elif l[0] == '\n':
                sents.append(this_sent)
            else:
                this_sent.append(tuple(l.split()))
    return sents

# 학습용 말뭉치 데이터 로딩
corpus = read_file('./data/chatbot/train.txt')
print(type(corpus), len(corpus))
print(corpus[:1])

<class 'list'> 3555
[[('1', '한편', 'NNG', 'O'), ('1', ',', 'SP', 'O'), ('2', 'AFC', 'SL', 'O'), ('2', '챔피언스', 'NNG', 'O'), ('2', '리그', 'NNG', 'O'), ('3', 'E', 'SL', 'B_OG'), ('3', '조', 'NNG', 'I'), ('3', '에', 'JKB', 'O'), ('4', '속하', 'VV', 'O'), ('4', 'ㄴ', 'ETM', 'O'), ('5', '포항', 'NNP', 'O'), ('6', '역시', 'MAJ', 'O'), ('7', '대회', 'NNG', 'O'), ('8', '8강', 'NNG', 'O'), ('9', '진출', 'NNG', 'O'), ('9', '이', 'JKS', 'O'), ('10', '불투명', 'NNG', 'O'), ('10', '하', 'VV', 'O'), ('10', '다', 'EC', 'O'), ('11', '.', 'SF', 'O')]]


In [14]:
# 2. 학습용데이터셋 생성 - corpus에서 단어와 BIO태그만 학습용데이터셋으로 사용
# ; 한편, AFC챔피언스리그 E조에 속한 포항 역시 대회 8강 진출이 불투명하다 .
# 1) 0번째 : 원본문장에서 분리된 단어 토큰들을 sentences에 저장
# 2) sentences의 단어 시퀀스에 해당하는 BIO태그정보들을 tags에 저장
# 3) E조 : E -> B_OG, 조 -> I로 태그
# 4) 단어시퀀스의 평균길이(34.039)를 기준으로 시퀀스패딩크기를 결정
sentences, tags = [], []
for t in corpus:
    tagged_sentences = []
    sentence, bio_tag = [], []
    for w in t:
        tagged_sentences.append((w[1], w[3]))
        sentence.append(w[1])
        bio_tag.append(w[3])
        
    sentences.append(sentence)
    tags.append(bio_tag)
    
print(f'샘플의 크기 : {len(sentences)}')
print(f'0번째 샘플문장의 시퀀스 : {sentences[0]}')
print(f'0번째 샘플 bio 태그 : {tags[0]}')
print(f'샘플문장 시퀀스의 최대길이  : {max(len(l) for l in sentences)}')
print(f'샘플문장 시퀀스의 평균길이  : {sum(map(len, sentences))/len(sentences)}')

샘플의 크기 : 3555
0번째 샘플문장의 시퀀스 : ['한편', ',', 'AFC', '챔피언스', '리그', 'E', '조', '에', '속하', 'ㄴ', '포항', '역시', '대회', '8강', '진출', '이', '불투명', '하', '다', '.']
0번째 샘플 bio 태그 : ['O', 'O', 'O', 'O', 'O', 'B_OG', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
샘플문장 시퀀스의 최대길이  : 168
샘플문장 시퀀스의 평균길이  : 34.03909985935302


In [35]:
# 3. 토크나이저정의
# 생성된 학습데이터셋에서 단어시퀀스와 태스시퀀스를 사전으로 등록하기 위해
# 토크나이저를 정의한 후 fit_on_texts()함수를 호출
# OOV : Out Of Vocabulary의 약자로 단어사전에 포함되지 않은 단어를 의미

# 1) 단어사전의 첫 번째 인덱스 토큰값을 OOV로 설정 및 토크나이징
sent_tokenizer = preprocessing.text.Tokenizer(oov_token='OOV') 
sent_tokenizer.fit_on_texts(sentences)
tag_tokenizer = preprocessing.text.Tokenizer(lower=False) # 태그정보는 소문자로 변환하지 않겠다.
tag_tokenizer.fit_on_texts(tags)

# 2) 생성된 사전 리스트(sent_tokenizer)를 이용해서 단어사전과 태그사전의 크기설정
vocab_size = len(sent_tokenizer.word_index) + 1;
tag_size = len(tag_tokenizer.word_index) + 1;
print(f'단어사전의 크기 = {vocab_size}')
print(f'BIO태그 단어사전의 크기 = {tag_size}')

# 3) 학습용 단어 시퀀스 생성
# sent_tokenizer 사전데이터를 시퀀번호형태로 인코딩처리
X_train = sent_tokenizer.texts_to_sequences(sentences)
y_train = tag_tokenizer.texts_to_sequences(tags)
print(X_train[0], y_train[0])
# y_train 1 = 'O', 3='B_OG', 2='I'

# 4) index to Word, index to NER정의
index_to_word = sent_tokenizer.index_word # 단어시퀀스인덱스를 단어로 변환, 183 -> '한편'
index_to_ner = tag_tokenizer.index_word # 태그시퀀스인덱스를 개체명으로 변환, 3 -> 'B_OG'
index_to_ner[0] = 'PAD'

# 5) 시퀀스패딩처리
# 개체명인식모델의 입출력벡터크기를 동일하게 설정하기 위해 시퀀스패딩작업을 진행
# max_len값은 단어시퀀스의 평균길이(39.039)보다 넉넉하게 40으로 정의
max_len = 40
X_train = preprocessing.sequence.pad_sequences(X_train, padding='post', maxlen=max_len)
y_train = preprocessing.sequence.pad_sequences(y_train, padding='post', maxlen=max_len)

# 6) 학습용 vs 테스트용 = 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=.2, random_state=0)

# 7) 출력데이터를 원핫인코딩
y_train = tf.keras.utils.to_categorical(y_train, num_classes=tag_size)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=tag_size)

# 8) 데이터정제결과출력
print(f'학습용 샘플 시퀀스 크기 = {X_train.shape}')
print(f'학습용 샘플 레이블 크기 = {y_train.shape}')
print(f'테스트 샘플 시퀀스 크기 = {X_test.shape}')
print(f'테스트 샘플 레이블 크기 = {y_test.shape}')

단어사전의 크기 = 13834
BIO태그 단어사전의 크기 = 8
[183, 11, 4276, 884, 162, 931, 402, 10, 2608, 7, 1516, 608, 145, 1361, 414, 4, 6347, 2, 8, 3] [1, 1, 1, 1, 1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
학습용 샘플 시퀀스 크기 = (2844, 40)
학습용 샘플 레이블 크기 = (2844, 40, 8)
테스트 샘플 시퀀스 크기 = (711, 40)
테스트 샘플 레이블 크기 = (711, 40, 8)


In [36]:
# 4. Bi-LSTM모델 정의
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

# 1) 개체인식모델을 순차모델방식으로 구현
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=30, input_length=max_len, mask_zero=True))
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.5, recurrent_dropout=0.25)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=10)

# 2) 평가결과
print(f'평가결과 : {model.evaluate(X_test, y_test)[1]}')


Epoch 1/10


23/23 [==============================] - 18s 357ms/step - loss: 0.6941 - accuracy: 0.8399
Epoch 2/10
23/23 [==============================] - 8s 332ms/step - loss: 0.3070 - accuracy: 0.9026
Epoch 3/10
23/23 [==============================] - 8s 329ms/step - loss: 0.2073 - accuracy: 0.9285
Epoch 4/10
23/23 [==============================] - 8s 334ms/step - loss: 0.1557 - accuracy: 0.9468
Epoch 5/10
23/23 [==============================] - 8s 332ms/step - loss: 0.1142 - accuracy: 0.9640
Epoch 6/10
23/23 [==============================] - 8s 350ms/step - loss: 0.0793 - accuracy: 0.9759
Epoch 7/10
23/23 [==============================] - 8s 340ms/step - loss: 0.0581 - accuracy: 0.9818
Epoch 8/10
23/23 [==============================] - 8s 362ms/step - loss: 0.0473 - accuracy: 0.9848
Epoch 9/10
23/23 [==============================] - 8s 360ms/step - loss: 0.0394 - accuracy: 0.9869
Epoch 10/10
23/23 [==============================] - 1s 20ms/step - loss: 0.2722 - accuracy: 0.9

###### 결과분석

* 정확도가 94%정도이지만 개체명인식에 사용되는 평가는 단순하게 평가결과로 성능평가를 하는 것이 아니라
* `f1-score로 성능평가를 해야 한다.`
* f1-score를 계산하기 위해서는 `정밀도와 재현율을 사용`해야 한다.
  - 정확도(Accuracy)  : 실제정답과 얼마나 유사한지
  - 정밀도(Precision) : `정밀도가 높으면 결과값이 일정하게 분포`되어 있다.
  - 재현율(Recall)    : `실제정답을 예측모델이 정답으로 예측한 비율`을 의미
  - 공식
    -   $$f1 score = 2 x \frac{정밀도 x 재현율}{정밀도 + 재현율}$$
  

In [54]:
# 시퀀스를 NER로 변환함수
def sequences_to_tag(sequences):
    result = []
    for sequence in sequences:
        temp = []
        for pred in sequence:
            pred_index = np.argmax(pred)
            temp.append(index_to_ner[pred_index].replace("PAD", "O"))
        result.append(temp)
    return result

In [55]:
# 테스트데이터셋의 NER예측
y_predicted = model.predict(X_test)
pred_tags = sequences_to_tag(y_predicted) # 예측 개체인식명 태그
test_tags = sequences_to_tag(y_test)      # 실제 개체인식명 태그

23/23 [==============================] - 0s 18ms/step


In [56]:
# f1-score계산
from seqeval.metrics import f1_score, classification_report
print(classification_report(test_tags, pred_tags))
print('f1-score: {:.2%}'.format(f1_score(test_tags, pred_tags)) )

              precision    recall  f1-score   support

           _       0.60      0.61      0.61       657
         _DT       0.92      0.92      0.92       335
         _LC       0.71      0.59      0.65       312
         _OG       0.67      0.58      0.62       481
         _PS       0.75      0.46      0.57       374
         _TI       0.89      0.76      0.82        66

   micro avg       0.71      0.63      0.67      2225
   macro avg       0.76      0.65      0.70      2225
weighted avg       0.72      0.63      0.66      2225

f1-score: 66.71%


In [58]:
# 문장예측
word_to_index = sent_tokenizer.word_index
new_sentence = '삼성전자 출시 스마트폰 오늘 애플 도전장 내밀다.'.split()
new_x = []

for w in new_sentence:
    try:
        new_x.append(word_to_index.get(w, 1))
    except KeyError:
        # 모르는 단어의 경우 OOV
        new_x.append(word_to_index['OOV'])
        
print(f'새로운 단어의 시퀀스 {new_x}')    

새로운 단어의 시퀀스 [531, 307, 1476, 286, 1507, 6766, 1]


In [60]:
# NER예측
new_padded_seqs = preprocessing.sequence.pad_sequences([new_x], padding='post', value=0, maxlen=max_len)
p = model.predict(np.array([new_padded_seqs[0]]))
p = np.argmax(p, axis=1) # 예측된 NER 인덱스값을 추출
print('{:10} {:5}'.format("단어", "예측된 NER"))
print('-'*60)
for w, pred in zip(new_sentence, p[0]):
    print('{:10} {:5}'.format(w, index_to_ner[pred]))

1/1 [==============================] - 0s 32ms/step
단어         예측된 NER
------------------------------------------------------------
삼성전자       B_TI 
출시         I    
스마트폰       B_DT 
오늘         PAD  
애플         B_TI 
도전장        B_OG 
내밀다.       B_TI 


In [ ]:
삼성전자       B_OG 
출시           O    
스마트폰       O 
오늘           B_DT  
애플           B_OG 
도전장         I 
내밀다.        I